In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from sklearn.metrics import accuracy_score
from PIL import Image
from regnet import RegNet, MODELS
import cv2

Print all available model names:

In [ ]:
print(list(MODELS.keys()))

In [ ]:
def eval_model(model, dataloader, device, criterion=None):
    loss_value = []
    y_pred = []
    y_true = []

    model.eval()
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            if out.size(1) == 1:
                # regression
                out = torch.squeeze(out, 1)

            if criterion is not None:
                loss = criterion(out, yb)
                loss_value.append(loss.item())

            y_pred.append(out.detach().cpu())
            y_true.append(yb.detach().cpu())

    if criterion is not None:
        loss_value = sum(loss_value) / len(loss_value)
        return torch.cat(y_pred), torch.cat(y_true), loss_value
    else:
        return torch.cat(y_pred), torch.cat(y_true)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device', device)

val_trainsforms = transforms.Compose([
    transforms.Resize(256, interpolation=Image.BICUBIC),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# set 'root' to path that contains following validation subset files for imagenet dataset:
# ILSVRC2012_devkit_t12.tar.gz
# ILSVRC2012_img_val.tar
# meta.bin
# To download ImageNet dataset: http://image-net.org/download
val_dataset = datasets.ImageNet(root="/path/to/imagenet/dataset/", split="val",
                                transform=val_trainsforms)

In [ ]:
myregnet_scores = {}
for key in MODELS.keys():
    model = RegNet(key, pretrained=True)
    model.to(device)

    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=32, shuffle=False,
        num_workers=2, pin_memory=True)
    
    y_pred, y_true = eval_model(model, val_loader, device)
    _, y_pred = torch.max(y_pred, 1)

    score = accuracy_score(y_pred, y_true)
    error = 1 - score
    print(f"{key}: Accuracy: {score:.4%}, Error: {error:.4%}")
    myregnet_scores[key] = score

Expected evaluation accuracy values on ImageNet validation set 

RegNetX-200MF: Accuracy: 69.7180%, Error: 30.2820%  
RegNetX-400MF: Accuracy: 73.4700%, Error: 26.5300%  
RegNetX-600MF: Accuracy: 74.7540%, Error: 25.2460%  
RegNetX-800MF: Accuracy: 75.6620%, Error: 24.3380%  
RegNetX-1.6GF: Accuracy: 77.6320%, Error: 22.3680%  
RegNetX-3.2GF: Accuracy: 78.8500%, Error: 21.1500%  
RegNetX-4.0GF: Accuracy: 79.0600%, Error: 20.9400%  
RegNetX-6.4GF: Accuracy: 79.5560%, Error: 20.4440%  
RegNetX-8.0GF: Accuracy: 79.7840%, Error: 20.2160%  
RegNetX-12GF: Accuracy: 80.1380%, Error: 19.8620%  
RegNetX-16GF: Accuracy: 80.5600%, Error: 19.4400%  
RegNetX-32GF: Accuracy: 80.7760%, Error: 19.2240%  
RegNetY-200MF: Accuracy: 71.3560%, Error: 28.6440%  
RegNetY-400MF: Accuracy: 74.9200%, Error: 25.0800%  
RegNetY-600MF: Accuracy: 76.1820%, Error: 23.8180%  
RegNetY-800MF: Accuracy: 77.1060%, Error: 22.8940%  
RegNetY-1.6GF: Accuracy: 78.6800%, Error: 21.3200%  
RegNetY-3.2GF: Accuracy: 79.5800%, Error: 20.4200%  
RegNetY-4.0GF: Accuracy: 80.1120%, Error: 19.8880%  
RegNetY-6.4GF: Accuracy: 80.5120%, Error: 19.4880%  
RegNetY-8.0GF: Accuracy: 80.6140%, Error: 19.3860%  
RegNetY-12GF: Accuracy: 81.0520%, Error: 18.9480%  
RegNetY-16GF: Accuracy: 81.1040%, Error: 18.8960%  
RegNetY-32GF: Accuracy: 81.4640%, Error: 18.5360%  